# Creating extensions using numpy and scipy

In this notebook, we shall go through two tasks:

1. Create a neural network layer with no parameters. 
  - This calls into **numpy** as part of it's implementation
2. Create a neural network layer that has learnable weights
  - This calls into **SciPy** as part of it's implementation

In [1]:
import torch
from torch.autograd import Function
from torch.autograd import Variable

## Parameter-less example

This layer doesn't particularly do anything useful or mathematically correct.

It is aptly named BadFFTFunction

**Layer Implementation**

In [2]:
from numpy.fft import rfft2, irfft2
class BadFFTFunction(Function):
    
    def forward(self, input):
        numpy_input = input.numpy()
        result = abs(rfft2(numpy_input))
        return torch.FloatTensor(result)
    
    def backward(self, grad_output):
        numpy_go = grad_output.numpy()
        result = irfft2(numpy_go)
        return torch.FloatTensor(result)

# since this layer does not have any parameters, we can
# simply declare this as a function, rather than as an nn.Module class
def incorrect_fft(input):
    return BadFFTFunction()(input)

**Example usage of the created layer:**

In [3]:
input = Variable(torch.randn(8, 8), requires_grad=True)
result = incorrect_fft(input)
print(result.data)
result.backward(torch.randn(result.size()))
print(input.grad)


  6.9997  11.0343   9.7395   6.0724   6.0526
  7.0250  11.4841   7.1110   5.6337   8.6441
  7.8062  10.9281   9.8279  23.4972   7.4842
  6.4962   4.5987   0.7936   3.9360   4.9595
  9.7913  10.3732   1.6261   2.0874  14.5295
  6.4962   5.7111   1.9793   8.8037   4.9595
  7.8062   8.7752   6.4442  14.1250   7.4842
  7.0250   5.4642   1.7983   4.4346   8.6441
[torch.FloatTensor of size 8x5]

Variable containing:
-0.0129  0.0330  0.0036 -0.0737  0.2354 -0.0737  0.0036  0.0330
 0.0542  0.0986 -0.0382 -0.1137 -0.0944 -0.0973 -0.0172 -0.0021
-0.1538 -0.1444  0.0356  0.1590  0.0588 -0.0188 -0.0611  0.0346
 0.1511  0.0370 -0.2513 -0.1518  0.1513 -0.2312 -0.0896 -0.1450
-0.1668 -0.0814  0.1954  0.1405  0.2191  0.1405  0.1954 -0.0814
 0.1511 -0.1450 -0.0896 -0.2312  0.1513 -0.1518 -0.2513  0.0370
-0.1538  0.0346 -0.0611 -0.0188  0.0588  0.1590  0.0356 -0.1444
 0.0542 -0.0021 -0.0172 -0.0973 -0.0944 -0.1137 -0.0382  0.0986
[torch.FloatTensor of size 8x8]



## Parametrized example

This implements a layer with learnable weights.

It implements the Cross-correlation with a learnable kernel.

In deep learning literature, it's confusingly referred to as Convolution.

The backward computes the gradients wrt the input and gradients wrt the filter.

**Implementation:**

*Please Note that the implementation serves as an illustration, and we did not verify it's correctness*

In [4]:
from scipy.signal import convolve2d, correlate2d
from torch.nn.modules.module import Module
from torch.nn.parameter import Parameter

class ScipyConv2dFunction(Function):
    
    def forward(self, input, filter):
        result = correlate2d(input.numpy(), filter.numpy(), mode='valid')
        self.save_for_backward(input, filter)
        return torch.FloatTensor(result)
    
    def backward(self, grad_output):
        input, filter = self.saved_tensors
        grad_input = convolve2d(grad_output.numpy(), filter.t().numpy(), mode='full')
        grad_filter = convolve2d(input.numpy(), grad_output.numpy(), mode='valid')
        return torch.FloatTensor(grad_input), torch.FloatTensor(grad_filter)


class ScipyConv2d(Module):
    
    def __init__(self, kh, kw):
        super(ScipyConv2d, self).__init__()
        self.filter=Parameter(torch.randn(kh, kw))
    
    def forward(self, input):
        return ScipyConv2dFunction()(input, self.filter)

**Example usage: **

In [5]:
module = ScipyConv2d(3, 3)
print(list(module.parameters()))
input = Variable(torch.randn(10, 10), requires_grad=True)
output = module(input)
print(output)
output.backward(torch.randn(8, 8))
print(input.grad)

[Parameter containing:
 0.0460  0.5052  0.9281
 0.8355  1.2642 -0.1283
 1.7027 -0.3146 -0.6927
[torch.FloatTensor of size 3x3]
]
Variable containing:
 1.4619 -4.0543  0.4391 -0.5423 -4.3719  3.9728 -0.4084 -2.8224
-3.6799 -3.9278  4.9932 -3.8952  3.0663  1.6303  2.9775  1.1806
-3.1694  2.1434  0.4432  1.6941  1.9344 -0.1196  1.1259  4.3571
-0.7934 -1.4610  2.2360  0.6406  0.3729  1.9140  0.2427  0.4298
-2.2961 -0.4189  5.6658  0.8090 -1.3030  2.2934  0.7164 -0.0272
 1.0649  1.0400 -1.3774 -0.2026 -0.9841  1.7192  3.0843  3.4241
 3.2743 -1.8780 -2.3084  0.8508  1.1622  0.6060  2.5559  1.0228
-2.3282 -1.1790 -2.4604 -1.9252 -1.3962  1.1054  3.6035  3.1302
[torch.FloatTensor of size 8x8]

Variable containing:
 0.0427  0.7780  1.7383  1.8333  3.8198  0.1135 -3.5576 -4.3994 -0.4354 -0.6021
 0.4661  1.2470  2.1080  6.3960  0.6894 -4.5144 -3.2005 -0.2762  0.3508  1.7803
 0.8492  0.9083  4.1836  0.6133 -3.4092 -1.8541  0.2254  3.6970  1.0382  0.5031
 0.0919  1.7864  1.5422  0.2942  2.0176  1.0